In [1]:
from scripts.parser import DNS_Shop_Parser
from keybert import KeyBERT
from pymystem3 import Mystem
from wordcloud import WordCloud
import scripts.text_processing as tp
from config import Config
# import multiprocessing as mp
# import nltk
import re
import os

In [2]:
# TODO многпоточная лемматизация,
# Добавить список стоп-слов

На вход приходит список словарей. Каждый словарь может содержать ключи 'Достоинства', 'Недостатки', 'Комментарий', 'Фото', а также ключи вида "Дополнение от 27 мая 2025 года", от которых необходимо избавиться.\
Наблюдение: если в отзыве есть "Дополнение...", то есть как минимум 'Достоинства' или 'Недостатки' или 'Комментарий'.

Препроцессинг текста

In [3]:
parser = DNS_Shop_Parser(headless=Config.BROWSER_HEADLESS)

Извлечение отзывов из текстового файла на время тестов, воизбежание бана в ДНС

In [4]:
reviews = []
with open(Config.RAW_HTML_PATH, 'r', encoding='utf-8') as file:
    reviews = parser.extract_reviews(file.read()) # propper form: list of dictionaries

In [5]:
# Извлечение отзывов задуманным способом, через открытие и скраппинга страницы
# parser.open_DNS_site() # execute once
# reviews = parser.get_product_reviews('https://www.dns-shop.ru/product/opinion/7ffd0f95a89cd21a/667-smartfon-xiaomi-redmi-note-14-256-gb-zelenyj/', 500) # propper form: list of dictionaries
# parser.parse_helper.page_to_txt(RAW_HTML_PATH)

Препроцессинг корпуса

In [6]:
# preprocessed_list = tp.preprocess_list_of_texts(tp.convert_to_list_of_reviews(reviews),to_lower=Config.ANALYZER_TO_LOWERCASE, erase_punct=Config.ANALYZER_ERASE_PUNCTUATION)

In [7]:
positive_list, negative_list = tp.split_positive_negative_parts(reviews, Config.ANALYZER_REVIEW_LEN_TRESHOLD)

In [8]:
positive_list = tp.preprocess_list_of_texts(positive_list, to_lower=Config.ANALYZER_TO_LOWERCASE, erase_punct=Config.ANALYZER_ERASE_PUNCTUATION)
negative_list = tp.preprocess_list_of_texts(negative_list, to_lower=Config.ANALYZER_TO_LOWERCASE, erase_punct=Config.ANALYZER_ERASE_PUNCTUATION)

In [9]:
model = KeyBERT(Config.ANALYZER_KEYBERT_MODEL_NAME)

Облако слов без лемматизации корпуса

In [10]:
positive_raw_keywords_list = tp.extract_keyphrases(positive_list, model, Config.ANALYZER_TOP_N_KEYWORDS, Config.ANALYZER_KEYPHRASE_NGRAM_RANGE)
positive_keywords_dict = tp.get_dict_keyphrases(positive_raw_keywords_list)
negative_raw_keywords_list = tp.extract_keyphrases(negative_list, model, Config.ANALYZER_TOP_N_KEYWORDS, Config.ANALYZER_KEYPHRASE_NGRAM_RANGE)
negative_keywords_dict = tp.get_dict_keyphrases(negative_raw_keywords_list)

In [ ]:
wordcloud = WordCloud(width=Config.WORDCLOUD_WIDTH,
                      height=Config.WORDCLOUD_HEIGHT,
                      background_color=Config.WORDCLOUD_BACKGROUND_COLOR,
                      margin=Config.WORDCLOUD_MARGIN,
                      colormap=Config.WORDCLOUD_COLORMAP,
                      collocations=Config.WORDCLOUD_COLLOCATIONS,
                      random_state=Config.WORDCLOUD_RANDOM_STATE)

In [12]:
wordcloud.generate_from_frequencies(positive_keywords_dict)
image = wordcloud.to_image()
# image.show()
image.save(os.path.join(Config.DATA_DIR, 'positive.png'))

In [13]:
wordcloud.generate_from_frequencies(negative_keywords_dict)
image = wordcloud.to_image()
# image.show()
image.save(os.path.join(Config.DATA_DIR, 'negative.png'))

Облако слов с лемматизацией корпуса

In [14]:
mystem = Mystem()

In [15]:
# При таком подходе лемматизация занимает чуть более 7 минут
# lemmatized_list = get_lemmatized_text(preprocessed_list, mystem)    # Необходима оптимизация + многопоточность. 
                                                                    # Другой вариант - сконкатенировать список в один большой текст и добавить разделитель. Далее по разделителю преобразовать в список

In [16]:
splitter = ' !@#$%^&*() ' # Костыльный разделитель
lemmatized_positive = ''.join(mystem.lemmatize(splitter.join(positive_list))) # мерджим в текст с разделителем splitter, лемматизируем -> list[str], снова мерджим
lemmatized_positive = lemmatized_positive.split(splitter)
lemmatized_negative = ''.join(mystem.lemmatize(splitter.join(negative_list)))
lemmatized_negative = lemmatized_negative.split(splitter)

In [17]:
positive_raw_keywords_list = tp.extract_keyphrases(lemmatized_positive, model, Config.ANALYZER_TOP_N_KEYWORDS, Config.ANALYZER_KEYPHRASE_NGRAM_RANGE)
positive_keywords_dict = tp.get_dict_keyphrases(positive_raw_keywords_list)
negative_raw_keywords_list = tp.extract_keyphrases(lemmatized_negative, model, Config.ANALYZER_TOP_N_KEYWORDS, Config.ANALYZER_KEYPHRASE_NGRAM_RANGE)
negative_keywords_dict = tp.get_dict_keyphrases(negative_raw_keywords_list)

In [ ]:
wordcloud = WordCloud(width=Config.WORDCLOUD_WIDTH,
                      height=Config.WORDCLOUD_HEIGHT,
                      background_color=Config.WORDCLOUD_BACKGROUND_COLOR, 
                      margin=Config.WORDCLOUD_MARGIN,
                      colormap=Config.WORDCLOUD_COLORMAP,
                      collocations=Config.WORDCLOUD_COLLOCATIONS,
                      random_state=Config.WORDCLOUD_RANDOM_STATE)

In [19]:
wordcloud.generate_from_frequencies(positive_keywords_dict)
image = wordcloud.to_image()
image.show()

In [20]:
wordcloud.generate_from_frequencies(negative_keywords_dict)
image = wordcloud.to_image()
image.show()